In [4]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.33.2 --progress-bar off
!pip install -qqq langchain==0.0.299 --progress-bar off
!pip install -qqq xformers==0.0.21 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq tokenizers==0.11.1 --progress-bar off


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-pubsublite 1.8.3 requires overrides<7.0.0,>=6.0.1, but you have overrides 7.7.0 which is incompatible.
jupyterlab 4.0.10 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [6]:
import pandas as pd
import json
!pip install langchain --upgrade

In [7]:
from huggingface_hub import login
login()

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [9]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

#MODEL_NAME = "mistralai/Mistral-7B-v0.1"
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.001})

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [11]:
print(llm)

HuggingFacePipeline
Params: {'model_id': 'gpt2', 'model_kwargs': {'temperature': 0.001}, 'pipeline_kwargs': None}


In [14]:
from langchain import PromptTemplate

template = """
<s>[INST] <<SYS>>
For a given user post sentence, does it show signs of the symptom. Answer in binary "yes" or "no",
for every symptom. The symptoms are as follows:
[Feeling nervous, anxious, or on edge, 

Not being able to stop or control worrying,

Worrying too much about different things, Trouble relaxing, 

Being so restless that it is hard to sit still, 

Becoming easily annoyed or irritable, 

Feeling afraid, as if something awful
might happen] .

Give the output as a python list of lists, with the list containing the symptom and the answer,
separated by a comma.
{text}:
<</SYS>>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [19]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
#from langchain.chains import SequentialDocumentsChain, LLMChain
from langchain.chains import LLMChain
examples = [
  {
    "question": """
I’ve been at February vacation away from school  this year and I can’t help but feel I’ve been wasting
it. I’ve done nothing of note, not even watching a movie or going to the grocery store, the only times 
I’ve left my house are for sports practice which I **dread** and just despise 
I think it’s been a long time coming but in the past few hours I’ve felt myself slipping in again 
to the pot of despair where I’m more irritated, testy, emotional and numb, and moody, and overall
a pit in my stomach forms that I have to live through. They’ve happened more and more and for 
longer durations since I began  taking meds and therapy. 
Any advice at all on how to buffer this dread and flare of depression would be greatly appreciated,
I can already feel myself sliding into it and just about all advice is appreciated.
""",
    "answer":
"""
[
          [
            "Feeling-nervous,-anxious,-or-on-edge ",
            "yes"
          ],
          [
            "Not-being-able-to-stop-or-control-worrying",
            "yes"
          ],
          [
            "Worrying-too-much-about-different-things",
            "yes"
          ],
          [
            "Trouble-relaxing",
            "no"
          ],
          [
            "Being-so-restless-that-it-is-hard-to-sit-still",
            "no"
          ],
          [
            "Becoming-easily-annoyed-or-irritable",
            "yes"
          ],
          [
            "Feeling-afraid,-as-if-something-awful-might-happen-soon",
            "yes"
          ]
        ]
"""},
    
  {
    "question": """
    When I'm sober I feel depressed, anxious, and sad at all times. Maybe sometimes I feel a little happiness or excitement on top of it all, but it last minutes, if not seconds.

I'm currently intoxicated and remembered briefly what I normally feel, thus my question. 

When I'm drunk or especially drunk AND high (weed) I feel so much better. I'm happy in the moment because I simply can't remember what I'm not happy about. Are there any negatives to being drunk and/or high at all times, other than while I'm working? Any reason to be sober?

I ask only because I can easily manage to be drunk and/or high on a very regular basis and I feel better than I have in months.
    """,
    "answer":
"""
[
          [
            "Feeling-nervous,-anxious,-or-on-edge ",
            "yes"
          ],
          [
            "Not-being-able-to-stop-or-control-worrying",
            "yes"
          ],
          [
            "Worrying-too-much-about-different-things",
            "yes"
          ],
          [
            "Trouble-relaxing",
            "no"
          ],
          [
            "Being-so-restless-that-it-is-hard-to-sit-still",
            "no"
          ],
          [
            "Becoming-easily-annoyed-or-irritable",
            "no"
          ],
          [
            "Feeling-afraid,-as-if-something-awful-might-happen-soon",
            "no"
          ]
        ]


"""
  }
]

In [20]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template= """For a given user post sentence, does it show signs of the symptom. Answer in binary "yes" or "no", for every symptom. The symptoms are as follows:
[Feeling nervous, anxious, or on edge, Not being able to stop or control worrying, Worrying too much about different things, Trouble relaxing, Being so restless that it is hard to sit still, Becoming easily annoyed or irritable, Feeling afraid, as if something awful
might happen] .

Give the output as a python list of lists, with the list containing the symptom and the answer, separated by a comma. \n Question: {question}\n{answer}""")

print(example_prompt.format(**examples[0]))

For a given user post sentence, does it show signs of the symptom. Answer in binary "yes" or "no", for every symptom. The symptoms are as follows:
[Feeling nervous, anxious, or on edge, Not being able to stop or control worrying, Worrying too much about different things, Trouble relaxing, Being so restless that it is hard to sit still, Becoming easily annoyed or irritable, Feeling afraid, as if something awful
might happen] .

Give the output as a python list of lists, with the list containing the symptom and the answer, separated by a comma. 
 Question: 
    I’ve been at February vacation away from school  this year and I can’t help but feel I’ve been wasting it. I’ve done nothing of note, not even watching a movie or going to the grocery store, the only times I’ve left my house are for sports practice which I **dread** and just despise. 

I think it’s been a long time coming but in the past few hours I’ve felt myself slipping in again to the pot of despair where I’m more irritated, t

In [24]:
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(few_shot_prompt.format(input=input))

For a given user post sentence, does it show signs of the symptom. Answer in binary "yes" or "no", for every symptom. The symptoms are as follows:
[Feeling nervous, anxious, or on edge, Not being able to stop or control worrying, Worrying too much about different things, Trouble relaxing, Being so restless that it is hard to sit still, Becoming easily annoyed or irritable, Feeling afraid, as if something awful
might happen] .

Give the output as a python list of lists, with the list containing the symptom and the answer, separated by a comma. 
 Question: 
    I’ve been at February vacation away from school  this year and I can’t help but feel I’ve been wasting it. I’ve done nothing of note, not even watching a movie or going to the grocery store, the only times I’ve left my house are for sports practice which I **dread** and just despise. 

I think it’s been a long time coming but in the past few hours I’ve felt myself slipping in again to the pot of despair where I’m more irritated, t

In [25]:
result = llm(prompt.format(text=input))
print(result)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


The symptoms you described indicate that your mother may be experiencing a severe depressive episode and may be at risk of suicide. Here are the answers to your question based on the symptoms you provided:

•She does not seem to be willing to socialize or engage in activities she used to enjoy. Yes, this is a common symptom of depression.
•She hates her job and her coworkers, but will not look for other employment. This could be due to feelings of hopelessness and lack of motivation, which are also common symptoms of depression.
•She sends nasty texts and emails to her siblings and cousins, accusing them of imagined insults. This could be a sign of irritability and agitation, which are also symptoms of depression.
•She drinks heavily and frequently makes statements about wanting to die. Yes, excessive alcohol consumption and suicidal ideation are common symptoms of depression.
•She believes she is getting bullied at work. This could be a sign of anxiety and paranoia, which are also com

In [26]:
from langchain.chains import LLMChain

# chain = LLMChain(llm=llm, prompt=prompt)

In [27]:
# %%time
# result = chain.run(text)
# print(result)

In [29]:
examples_chain = LLMChain(llm=llm, prompt=few_shot_prompt)

In [36]:
# with open('/kaggle/input/primate/primate_dataset.json', 'r') as file:
#     data = json.load(file)

# Initialize lists for columns
post_titles = []
second_columns = []
i=0
# Process each entry in the JSON data

import pandas as pd
df = pd.read_csv('/kaggle/input/prompt-dataset/prompt_dataset_primate.csv')

for index, row in df.iterrows():
    # Extract post_title and post_text
    print(index)
    post_title = row['Title']
    post_text = row['Text']
    print(f'TITLE: {post_title}')
    #print(f'TEXT: {post_text}')

    # Process post_text using the get_ans function
    # result = multi_chain.run
    #second_column_value = llm(prompt.format(text=post_text))
    post_text = post_text.replace("\n", " ")
    second_column_value = llm(few_shot_prompt.format(input=post_text))

    # Append values to lists
    post_titles.append(post_title)
    second_columns.append(second_column_value)

# Create a DataFrame
df = pd.DataFrame({'Title': post_titles, 'Output': second_columns})

# Write DataFrame to CSV
#df.to_csv('abc.csv', index=False)
print('SUCCESS')

0
TITLE: How am I supposed to improve my life when I constantly want to die
1
TITLE: Should I click yes to give the psychiatrist permission to share my information with one of my parents? What are they gonna share?
2
TITLE: What would you do in my position?
3
TITLE: SO says it's depression linked. How to stop being too much?
4
TITLE: Your experience with SSRI's
5
TITLE: Two Faced People - Need Advice
6
TITLE: How can I force myself to do important things when I’m feeling like shit?
7
TITLE: I feel like i'm loosing my best friend and I don't know what to do
8
TITLE: I hate myself so much, I don't deserve to live


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


9
TITLE: My reward system is fked


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


10
TITLE: My dad just threatened me for crying and going to my room and i would really need some kind words


...

38
TITLE: "Paying for happiness" - anyone else?


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


39
TITLE: What can I do…


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


SUCCESS


In [63]:
# df.to_csv('llama2-gad-fewshot.csv', index=False)
# print('SUCCESS')

SUCCESS
